In [33]:

from SSD_generate_anchors import generate_ssd_priors
from CustomDataLoader import DataAugmentor
from utility import *
from pathlib import Path
import collections
from ssd_loss_function_v2 import CustomLoss
from ssd300_model import SSD300
from keras import backend as K
import tensorflow as tf

In [2]:
root = Path.home()/'data'/'VOCdevkit'/'VOC2007'
# root  = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'/'VOC2007'

voc_2007_datafile = root/'ImageSets'/'Main'/'train.txt'
voc_2007_images   = root/'JPEGImages'
voc_2007_annotations = root/'Annotations'

In [3]:
SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

# the SSD orignal specs
specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

In [4]:
priors = generate_ssd_priors(specs)

batch_size = 8

testloader = DataAugmentor()
data       = testloader.flow_from_directory(root = root,
                                            data_file=voc_2007_datafile,
                                            target_size=300,
                                            batch_size = batch_size,
                                            shuffle    = True,
                                            num_classes = 21,
                                            priors      = priors
                                            )

In [5]:
sample = data[0]

images, targets = sample
print(images.shape)
print(targets.shape)

(8, 300, 300, 3)
(8, 8732, 5)


In [6]:
model = SSD300(input_shape=(300,300, 3), num_classes=21)

prediction = model.predict(images)

W0802 22:28:55.484102 139756464920384 deprecation_wrapper.py:119] From /home/puru/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 22:28:55.555014 139756464920384 deprecation_wrapper.py:119] From /home/puru/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0802 22:28:55.677284 139756464920384 deprecation_wrapper.py:119] From /home/puru/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [7]:
SSDLoss = CustomLoss(anchors   = priors, 
                     alpha     = 1.0
                    )

loc_data, positives = SSDLoss(y_true = targets, y_pred = prediction)

In [8]:
pos = K.cast(positives, dtype='uint8')

In [9]:
num_pos = K.sum(pos, axis=-1, keepdims=True)
print(num_pos)
print(K.eval(num_pos))

Tensor("Sum:0", shape=(8, 1), dtype=uint8)
[[ 41]
 [ 52]
 [100]
 [  7]
 [ 26]
 [ 19]
 [ 78]
 [ 18]]


In [10]:
pos_shape = K.shape(pos)
print(K.eval(pos_shape))
positives = K.reshape(positives, shape = (pos_shape[0], pos_shape[1],1))
pos_idx   = K.repeat_elements(positives, 4, axis=-1)

[   8 8732]


In [11]:
print(K.eval(K.shape(pos_idx)))

[   8 8732    4]


In [12]:
print(K.eval(K.shape(loc_data)))

[   8 8732    4]


In [22]:
pos_idx_ = K.cast(pos_idx, 'int32')

In [43]:
loc_p = K.tf.boolean_mask(loc_data, pos_idx)

In [45]:
loc_p = K.reshape(loc_p, shape= (-1, 4))

In [46]:
print(K.eval(K.shape(loc_p)))

[341   4]
